# Import packages

In [ ]:
import pandas as pd
import numpy as np
import torch
from lightning import pytorch as pl

from chemprop import data, featurizers, models

# Change model input here

In [ ]:
checkpoint_path = '../tests/data/example_model_v2.ckpt' # path to the checkpoint file. 
# If the checkpoint file is generated using the training notebook, it will be in the `checkpoints` folder with name similar to `checkpoints/epoch=19-step=180.ckpt`.

## Load model

In [ ]:
mpnn = models.MPNN.load_from_checkpoint(checkpoint_path)
mpnn

# Change predict input here

In [ ]:
test_path = '../tests/data/smis.csv'
smiles_column = 'smiles'

## Load test smiles

In [ ]:
df_test = pd.read_csv(test_path)
df_test

## Get smiles

In [ ]:
smis = df_test[smiles_column]
smis

## Get molecule datapoints

In [ ]:
test_data = [data.MoleculeDatapoint.from_smi(smi) for smi in smis]

## Get molecule dataset

In [ ]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
test_dset = data.MoleculeDataset(test_data, featurizer=featurizer)
test_loader = data.MolGraphDataLoader(test_dset, shuffle=False)

# Set up trainer

In [ ]:
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

In [ ]:
test_preds = np.concatenate(test_preds, axis=0)
df_test['pred'] = test_preds
df_test